In [5]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
import ast 

In [6]:
credits_df = pd.read_csv('/Users/danishabbasi/ML Projects (Python)/credits.csv')
movies_df = pd.read_csv('/Users/danishabbasi/ML Projects (Python)movies.csv')



FileNotFoundError: [Errno 2] No such file or directory: '/Users/danishabbasi/credits.csv'

In [ ]:
credits_df

In [ ]:
movies_df

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
credits_df


In [ ]:
movies_df

In [ ]:
credits_df.head()

In [ ]:
movies_df.tail()


In [ ]:
movies_df = movies_df.merge(credits_df, on='title')

In [ ]:
movies_df.head()

In [ ]:
movies_df.info()


In [ ]:
movies_df = movies_df[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]


In [ ]:
movies_df.head()

In [ ]:
movies_df.info()


In [ ]:
movies_df.isnull().sum()

In [ ]:
movies_df.dropna(inplace=True)

In [ ]:
movies_df.duplicated().sum()

In [ ]:
movies_df.iloc[0].genres


In [ ]:
def convert_formats(target_data):
    '''converts complex formats for data within each column in movies_df into simple data'''
    new_list = []
    for x in ast.literal_eval(target_data): 
        new_list.append(x['name'])
    return new_list

In [ ]:
movies_df['genres'] = movies_df['genres'].apply(convert_formats) # converts the data in genres column for use

In [ ]:
movies_df.head()

In [ ]:
movies_df['keywords'] = movies_df['keywords'].apply(convert_formats) # converts data in keywords column for use

In [ ]:
movies_df.head()

In [ ]:
def cast_scraper(target_data):
    '''scrapes data from cast column and returns first three cast members'''
    new_list = []
    counter = 0
    for x in ast.literal_eval(target_data):
        if counter != 3:
            new_list.append(x['name'])
        else:
            break
    return new_list

In [ ]:
movies_df['cast'] = movies_df['cast'].apply(cast_scraper) # sets cast column data to first three cast members

In [ ]:
movies_df.head()

In [ ]:
def retrieve_director(target_data):
    '''retrieves director from each movie in movies_df and sets to crew column data'''
    new_list = []
    for x in ast.literal_eval(target_data):
        if x['job'] == 'Director':
            new_list.append(x['name'])
    return new_list

In [ ]:
movies_df['crew'] = movies_df['crew'].apply(retrieve_director) # sets crew data column to director names 

In [ ]:
movies_df.head()

In [ ]:
movies_df['overview'] = movies_df['overview'].apply(lambda x:x.split())

In [ ]:
movies_df.head()

In [ ]:
movies_df['genres'] = movies_df['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])
movies_df['cast'] = movies_df['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
movies_df['crew'] = movies_df['crew'].apply(lambda x: [i.replace(" ", "") for i in x])

# removes the spaces in each column data to help recommender system not get confused

# for example, if an indiviudal wants James Cameron movies, to the recommender system 'James' and 'Cameron' are
# two seperate entities and the system will get confused if another director in the data has the name 'James'
# by removing the space in 'James Cameron' to 'JamesCameron' it will be one single entity and the recommender system
# will know which specific movie to recommender based on director
                                                                 

In [ ]:
movies_df.head()

In [ ]:
movies_df['tags'] = movies_df['overview'] + movies_df['genres'] + movies_df['keywords'] + movies_df['cast'] + movies_df['crew']

In [ ]:
movies_df.head()

In [ ]:
new_df = movies_df[['movie_id', 'title', 'tags']]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df.head()

In [ ]:
cv = CountVectorizer(max_features=5000, stop_words = 'english')

In [ ]:
movie_vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
cv.get_feature_names()

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

# using stemmer to eliminate similar words like "action" and "actions" from tags data

In [ ]:
def stemmer(text):
    '''function that uses text stemmer from nltk PorterStemmer'''
    x = []
    
    for i in text.split():
        x.append(ps.stem(i))
        
    return " ".join(x)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stemmer)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
movies_similarity = cosine_similarity(movie_vectors)

In [ ]:
movies_similarity

In [ ]:
def recommend(movie):
    ''''recommends five movies to user based on movie passed through function'''
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = movies_similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
    
    for x in movies_list:
        print(new_df.iloc[x[0]].title)

In [ ]:
recommend('Batman Begins')

In [ ]:
recommend('Batman Begins')